# Building Makemore Part 1 - Exercise Solutions  

## Ex1: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss. Did it improve over a bigram model?   

In [2]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-09-21 21:46:14--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.1s    

2024-09-21 21:46:15 (1.98 MB/s) - ‘names.txt’ saved [228145/228145]



In [3]:
# Preparing our list of words
words = open('names.txt', 'r').read().splitlines()

In [4]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
# Create our "string-to-index" dictionary
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
import torch

# create the training dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    iy = stoi[ch3]
    xs.append([ix1, ix2])
    ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [8]:
# Here be aware that contrary to what happens in Andrej's lecture on bigrams
# where `xs` is a 1-dim tensor, `xs.nelement()` won't return the number of
# training examples, but the total number of elements in `xs` (in our case two
# times the number of training examples). Use `.size(0)` or `.shape[0]` instead.
num = xs.size(0)
print('number of elements in our training dataset: ', num)

number of elements in our training dataset:  196113


In [9]:
xs.shape, ys.shape

(torch.Size([196113, 2]), torch.Size([196113]))

In [10]:
xs[:4], ys[:4]

(tensor([[ 0,  5],
         [ 5, 13],
         [13, 13],
         [13,  1]]),
 tensor([13, 13,  1,  0]))

In [11]:
# We will train a trigram language model using a neural network with a single layer
# of 27 neurons (one neuron per class, i.e for each character) each receiving
# now 27 + 27 = 54 inputs, instead of 27 inputs for the bigram model, since now
# a bigram starting a trigram will be encoded into a 54-dim vector.
# Indeed, a pair of indices (i,j) will be encoded into a 54-dim vector with a 1
# at the i-th position and also at the (27+j)-th position and 0 elsewhere.

# `xs` is a matrix, i.e. a 2-dim tensor, with 196113 rows and 2 columns.
xs.shape

torch.Size([196113, 2])

In [12]:
# one-hot encoding of `xs` will make it into a 3D tensor of size (196113, 2, 27)
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float()
xenc.shape

torch.Size([196113, 2, 27])

In [13]:
xenc.view(-1, 27*2).shape

torch.Size([196113, 54])

In [14]:
# To input `xs` to our network, we need to flatten it into a (196113, 54)-matrix.
# This way, we will have encoded an element of our training dataset, i.e. a pair
# of indices encoding a bigram, into a 54-dim vector (versus a 27-dim vector in the
# case of the bigram model in Andrej's lecture).

In [15]:
# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27+27, 27), generator=g, requires_grad=True)

In [16]:
# Sampling before training (spoiler: It's terrible!)

for i in range(20):

  out = []
  ix1 = 0
  ix2 = 0
  while True:

    xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
    xenc = xenc.view(-1, 27*2)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))

owo.
ziipijpgcdhiupgbdnnbxutfkhltr.
anfspzxsshcdhk.
zogvkbdkztaotadhkpgjppgxktwdnu.
bxoemziadbjmqadfkcnbjmuthsdziwlvp.
cochvxyedvpumyyftbdqaawlfyfwbelapxsshpgaocfvpgllbnvlenadsdziu.
zpgcbjpqgqqo.
.
ozikqllonkdhaqxqztiu.
nyvwhuhochkvlmybdujyigemcgjxqaongqlroogdppmcrhmydhvshakoxgjllopgjdzqaydgjpghfiupfydtawdzrpbxtanjpkxtstzxyetdgjkjxqfspwgduiiiadtdgjmpgjlmyftygiziunygnroenrhadljpgmkxgqmpfdhsdztdxqjrqllvwwd.
zxsjibdbjfbjkbdhjxxshusztaddpgcs.
.
kodziitziidaodch.
to.
nr.
gfnvabjxatlvapfm.
zksavdyktsdhadyqwwetdpmodrkwrt.
oennhmydukfk.
nnwpgfyddzqmraiusstaduvc.
zpionz.


In [17]:
W.shape

torch.Size([54, 27])

In [18]:
W

tensor([[ 1.5674, -0.2373, -0.0274,  ..., -0.0707,  2.4968,  2.4448],
        [-0.6701, -1.2199,  0.3031,  ...,  0.8032,  0.5411, -1.1646],
        [ 0.1476, -1.0006,  0.3801,  ..., -0.6279,  0.0770, -1.1641],
        ...,
        [ 0.5283, -0.9056, -0.0124,  ..., -0.9310, -0.0919,  0.1651],
        [-0.7125,  0.6541,  0.8071,  ..., -1.1854,  1.0008,  0.9374],
        [-0.2512, -0.8699,  0.5397,  ...,  0.0908, -0.4618, -0.8567]],
       requires_grad=True)

In [19]:
#---------------Our Actual Network------------------

# gradient descent
for k in range(200): # 200 iterations of the gradient descent

  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  xenc = xenc.view(-1, 27*2) # flatten to input to the network
  logits = xenc @ W # predict log-counts, after matrix multiplication we get a (num, 27)-matrix
  counts = logits.exp() # counts
  probs = counts / counts.sum(1, keepdims=True) # `probs` is a matrix,
                                                # `probs[torch.arange(num), ys]` below
                                                # selects for each example the prob assigned by the model to the last character of the trigram
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean() # the fst term computes the average negative log-likelihood loss for the current batch of data
                                                                         # the snd term is a L2 regularization, aka weight decay, that penalizes large weights
                                                                         # depending of the strenght parameter, here 0.01.
  if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update weights
  W.data -= 50 * W.grad

0: 4.1960
10: 2.5104
20: 2.3861
30: 2.3394
40: 2.3152
50: 2.3007
60: 2.2912
70: 2.2844
80: 2.2794
90: 2.2755
100: 2.2724
110: 2.2699
120: 2.2679
130: 2.2662
140: 2.2647
150: 2.2635
160: 2.2624
170: 2.2615
180: 2.2607
190: 2.2599


In [20]:
W

tensor([[-3.1582,  2.5806,  0.2739,  ..., -1.0979,  0.9909,  0.8251],
        [ 1.2707,  0.4196, -0.2165,  ..., -1.5259, -0.1087, -0.2081],
        [ 0.4628,  1.3754, -0.0873,  ..., -0.4159,  0.6824, -1.1060],
        ...,
        [ 0.5852,  0.4906, -0.4283,  ...,  0.6958,  0.2371,  0.0636],
        [ 1.5921,  1.9269, -0.6454,  ..., -0.9071, -1.6581,  0.0755],
        [ 0.3606,  2.1695, -0.3000,  ..., -0.3426,  1.5105,  0.3133]],
       requires_grad=True)

In [21]:
# last, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix1 = 0
  ix2 = 0
  while True:

    xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
    xenc = xenc.view(-1, 27*2)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))

aunide.
aliasad.
ushfay.
ainn.
aui.
ritoleras.
get.
usannaauranileniassibdainrwi.
ol.
seisiely.


## Ex2: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [ ]:
# build the dataset
block_size = 2 # context length: how many characters do we take to predict the next one

def build_dataset(words):
  X, Y = [], [] # `X` will store the input data and `Y` the labels
  for w in words:

    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X, Y

In [ ]:
X, Y = build_dataset(words)
X.shape, Y.shape

(torch.Size([228146, 2]), torch.Size([228146]))

In [ ]:
X[:5], Y[:5]

(tensor([[ 0,  0],
         [ 0,  5],
         [ 5, 13],
         [13, 13],
         [13,  1]]),
 tensor([ 5, 13, 13,  1,  0]))

In [ ]:
import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words)) # we require `int()` to convert the float `0.8*len(words)` back into a natural number that can be used below as an index
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [ ]:
Xtr.shape, Ytr.shape

(torch.Size([182625, 2]), torch.Size([182625]))

In [ ]:
Xdev.shape, Ydev.shape

(torch.Size([22655, 2]), torch.Size([22655]))

In [ ]:
Xte.shape, Yte.shape

(torch.Size([22866, 2]), torch.Size([22866]))

In [ ]:
# Reinitialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27+27, 27), generator=g, requires_grad=True)

In [ ]:
# train the model on the training set

# gradient descent
for k in range(200): # 200 iterations of the gradient descent

  # forward pass
  xenc = F.one_hot(Xtr, num_classes=27).float() # input to the network: one-hot encoding
  xenc = xenc.view(-1, 27*2) # flatten to input to the network
  logits = xenc @ W # predict log-counts, after matrix multiplication we get a (num, 27)-matrix
  counts = logits.exp() # counts
  probs = counts / counts.sum(1, keepdims=True)
  numtr = len(Xtr)
  loss = -probs[torch.arange(numtr), Ytr].log().mean() + 0.01*(W**2).mean()
  if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update weights
  W.data -= 50 * W.grad

0: 2.3581
10: 2.3572
20: 2.3566
30: 2.3787
40: 2.3885
50: 2.3769
60: 2.3875
70: 2.3766
80: 2.3868
90: 2.3763
100: 2.3864
110: 2.3760
120: 2.3860
130: 2.3757
140: 2.3857
150: 2.3755
160: 2.3855
170: 2.3753
180: 2.3853
190: 2.3752


In [ ]:
def loss(X, Y, W):
    xenc = F.one_hot(X, num_classes = 27).float()
    xenc = xenc.view(-1, 27*2)

    # probs is softmax
    logits = xenc @ W
    counts = torch.exp(logits)
    probs = counts / counts.sum(dim = 1, keepdim = True)

    # loss (normalized negative log likelihood)
    loss = - probs[torch.arange(len(X)), Y].log().mean() + 0.01*(W**2).mean()

    return loss.item()

In [ ]:
print(f"Train Loss: {loss(Xtr, Ytr, W):.4f}")
print(f"Dev Loss: {loss(Xdev, Ydev, W):.4f}")
print(f"Test Loss: {loss(Xte, Yte, W):.4f}")

Train Loss: 2.3851
Dev Loss: 2.3837
Test Loss: 2.3897


No significant variance between train/dev/test sets, so no sign of overfitting or underfitting. Can we fine-tune the parameters during the dev phase to slightly improve performance?

In [ ]:
# sample

g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix1 = 0
  ix2 = 0
  while True:

    xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
    xenc = xenc.view(-1, 27*2)
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))

juwide.
janasad.
alen.
amainn.
kai.
ritoleras.
tee.
adannaauranileniassibdainrwi.
ta.
saisiely.


## Ex3: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [ ]:
# fine-tune the parameters

# gradient descent
for k in range(20):

  # forward pass
  xenc = F.one_hot(Xdev, num_classes=27).float()
  xenc = xenc.view(-1, 27*2)
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  numdev = len(Xdev)
  loss = -probs[torch.arange(numdev), Ydev].log().mean() + 0.01*(W**2).mean()
  print(f"{k}: {loss.item():.4f}")

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update weights
  W.data -= 50 * W.grad



0: 2.3447
1: 2.3437
2: 2.3430
3: 2.3423
4: 2.3417
5: 2.3411
6: 2.3406
7: 2.3402
8: 2.3398
9: 2.3394
10: 2.3391
11: 2.3387
12: 2.3384
13: 2.3382
14: 2.3379
15: 2.3377
16: 2.3375
17: 2.3375
18: 2.3379
19: 2.3395


I tried various regularization strengths, 0.01 seems to give the best results.

In [ ]:
# We take the best setting of the smoothing and evaluate on the test set once more

xenc = F.one_hot(Xte, num_classes=27).float()
xenc = xenc.view(-1, 27*2)
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
numte = len(Xte)
loss = -probs[torch.arange(numte), Yte].log().mean() + 0.001*(W**2).mean()
print(f"Test Loss: {loss.item():.4f}")

Test Loss: 2.3605


We decreased a bit our loss wrt the one on the dev set.    

## Ex4: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

In [ ]:
# case of bigrams

# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
xs.shape

torch.Size([228146])

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
W[xs].shape

torch.Size([228146, 27])

In [ ]:
# gradient descent
for k in range(10):

  # forward pass
  logits = W[xs] # we directly index into W instead of using one_hot encoding
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(len(xs)), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  # backward pass
  W.grad = None
  loss.backward()

  # update
  W.data += -50 * W.grad



3.7686190605163574
3.3787858486175537
3.1610772609710693
3.027181625366211
2.9344804286956787
2.8672285079956055
2.81665301322937
2.777146100997925
2.745253562927246
2.7188308238983154


In [ ]:
# we come back to the case of bigrams

# create the training dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    iy = stoi[ch3]
    xs.append([ix1, ix2])
    ys.append(iy)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
# Reinitialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27+27, 27), generator=g, requires_grad=True)

In [ ]:
xs.shape

torch.Size([196113, 2])

In [ ]:
xs[:,0].shape # is the tensor (1d-tensor, i.e. vector) of all the first elements of our 196,113 pairs of indices in our training dataset

torch.Size([196113])

In [ ]:
W[xs[:,0]].shape # is thus the lines of W corresponding to each of the indices in xs[:,0]

torch.Size([196113, 27])

In [ ]:
# gradient descent
for k in range(200):

  # forward pass
  logits = W[xs[:,0]] + W[27 + xs[:,1]] # we directly index into W insteaf of using one_hot encoding
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(len(xs)), ys].log().mean() + 0.01*(W**2).mean()
  if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update weights
  W.data -= 50 * W.grad

0: 4.1960
10: 2.5104
20: 2.3861
30: 2.3394
40: 2.3152
50: 2.3007
60: 2.2912
70: 2.2844
80: 2.2794
90: 2.2755
100: 2.2724
110: 2.2699
120: 2.2679
130: 2.2662
140: 2.2647
150: 2.2635
160: 2.2624
170: 2.2615
180: 2.2607
190: 2.2599


## Ex5: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?

In [ ]:
# Reinitialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27+27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(200):

  # forward pass
  logits = W[xs[:,0]] + W[27 + xs[:,1]] #predict log-counts, directly indexing into W
  loss = F.cross_entropy(logits, ys) + 0.01*(W**2).mean() # choose cross entropy as the loss function instead of NLL
  if k % 10 == 0:
        print(f"{k}: {loss.item():.4f}")

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update weights
  W.data -= 50 * W.grad

0: 4.1960
10: 2.5104
20: 2.3861
30: 2.3394
40: 2.3152
50: 2.3007
60: 2.2912
70: 2.2844
80: 2.2794
90: 2.2755
100: 2.2724
110: 2.2699
120: 2.2679
130: 2.2662
140: 2.2647
150: 2.2635
160: 2.2624
170: 2.2615
180: 2.2607
190: 2.2599


Keeping the same learning rate, weigth decay and training dataset `xs`, we achieve the same result with cross entropy. However, the code is slightly simplified, since `F.cross_entropy` automatically applies a softmax to convert logits into probabilities. Also, combining softmax and log probabilities into a single step reduces the risk of numerical instability.  